# TSFS12: Spawn a vehicle and attach a lidar

Small example how to attach a sensor, in this case a lidar. For more details, look in the example code ```manual_control.py```.

In [1]:
import carla
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


## Connect to a running Carla server and get main pointers

Get pointer to Carla and load a world

In [3]:
HOST = 'localhost'
PORT = 2000
client = carla.Client(HOST, PORT)
client.set_timeout(2.0)

Get pointers to the world and blueprint library (templates to all actors and vehicles).

In [4]:
world = client.get_world()
bpl = world.get_blueprint_library()
sp = world.get_spectator()  # Get pointer to the spectator, i.e., camera in the server window

Get information about the spectator, e.g., position and orientation. Experiment by moving around in the world and see how things change.

### Spawn a car in the world

Find a random type of Audi in the library of available vehicles.

In [5]:
print('Available Audis')
for c in bpl.filter('vehicle.audi.*'):
    print(c.id)
bp = random.choice(bpl.filter('vehicle.audi.*'))
print('Random selection: {}'.format(bp.id))

Available Audis
vehicle.audi.a2
vehicle.audi.etron
vehicle.audi.tt
Random selection: vehicle.audi.a2


Spawn a vehicle at the position of the spectator

In [6]:
actors = []
tf = sp.get_transform()
tf.location.z = 1
car = world.spawn_actor(bp, carla.Transform(tf.location))
actors.append(car)

In [7]:
tf.location.z += 15
sp.set_transform(tf)

## Lidar

Attach a lidar sensor

In [8]:
lidar_bpl = bpl.filter('sensor.lidar.ray_cast')[0]
lidar_bpl.set_attribute('range', '5000')
lidar = world.spawn_actor(lidar_bpl, 
                  carla.Transform(carla.Location(x=2.5, z=1), 
                                  carla.Rotation(pitch=0)),
                  attach_to=car)
actors.append(lidar)

Define a simple callback and start listining. Don't listen too long since it accumulates all data.

In [9]:
lidar_data = []
def lidar_callback(data):
        lidar_data.append(data)
lidar.listen(lidar_callback)

Stop listening

In [10]:
lidar.stop()
print('Number of lidar frames recorded: {}'.format(len(lidar_data)))

Number of lidar frames recorded: 853


Make a simple plot where all recorded points are in the same plot.

In [11]:
plt.figure(10, clear=True)
for di in lidar_data:
    points = np.frombuffer(di.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (int(points.shape[0] / 3), 3))
    h = points[:, 2] > 0
    plt.plot(points[h, 0], points[h, 1], 'k.', markersize=0.3)
_ = plt.axis([-10, 10, -10, 10])

Destroy all actors

In [12]:
for a in actors:
    a.destroy()